<a href="https://colab.research.google.com/drive/1RMcnOWQYgpAIE5Ti_MuVuLpnYuCR1N8Y?usp=sharing">
  <img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install transformers
!pip install sentencepiece
#!pip install tensorflow==2.4.0 

In [ ]:
# import module
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import nltk
nltk.download("stopwords")
AUTO = tf.data.experimental.AUTOTUNE
MAX_LENGTHS = 50

NUM_CATEGORICAL = 3
dfResult = pd.DataFrame(columns=[ "rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","ModelName"])
HISTORY_TRANSFORMERS = []

IMPORTING ALL DATASETS

In [ ]:
!gdown --id 1dhdMKM9akp0krL9ObL-kn8K9gSVOwPUp
!gdown --id 1160UzrabHFjB4QjLs6IWU4mZ6fZroGNf
!gdown --id 10FbqBhDzGl_JsFLxFI8SvWmouc0FWQtd

## Yow Read the Dataset

In [ ]:
!pip install openpyxl

In [ ]:
dfAllData = pd.read_csv("All Data Undersampling Max 200.csv")

In [ ]:
sample = dfAllData.loc[1,"tweet"]
def cleanIt(text):
  
  text = str(text)
  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 

  text = re.sub(text_cleaning_hash, " ",text).rstrip()
  text = re.sub(text_cleaning_re, " ",text).rstrip()
  
  return text
cleanIt(sample)

In [ ]:
stopword = nltk.corpus.stopwords.words('indonesian')
def stopWord(text):
  text = text.split()
  return " ".join([t for t in text if t not in stopword])
stopWord("bilamana apabila saya  ")

In [ ]:
dfNormal = pd.read_csv("Normal.csv")
dfAllData = dfAllData[dfAllData["labels"] != 0]


In [ ]:
dfAllData.loc[dfAllData["labels"] == 3,"labels"]= 2

In [ ]:
dfNormal.loc[:,"labels"] = 0

In [ ]:
dfAllData = pd.concat([dfAllData,dfNormal])

In [ ]:
dfAllData["tweet"] = dfAllData["tweet"].apply(cleanIt)
dfAllData["tweet"] = dfAllData["tweet"].apply(stopWord)
dfAllData = dfAllData[dfAllData["tweet"].apply(lambda x : len(x.rstrip().split()) > 4) == True ]
dfAllData[dfAllData["tweet"].apply(lambda x : len(x.split()) < 4) == True]

In [ ]:
for i in range(5):
    dfAllData = dfAllData.sample(frac=1).reset_index(drop=True)

In [ ]:
dfAllData

In [ ]:
import plotly.graph_objects as go
counted = dfAllData.groupby('labels')["labels"].aggregate("count")
print(counted)
fig = go.Figure()
fig.add_traces(go.Bar(y = ["Normal","Berita Bohong","HateSpeech","Offensive"],x = counted,orientation="h"))
fig.show()

## TransferLearning Section menggunakan transformer model

In [ ]:
# Check panjang text Narasi
from collections import Counter
seq_len = [len(text.split()) for text in dfAllData["tweet"] if type(text) != float]
print(Counter(seq_len))

In [ ]:
plt.figure(figsize=(18,12))
sns.histplot(list(dict(Counter(seq_len)).values()))

In [ ]:
METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
  ]

In [ ]:
WORD_SIZE = 5000
def dataForLstm():
  from tensorflow.keras.preprocessing.text import Tokenizer
  tokenizer = Tokenizer(num_words = WORD_SIZE)
  tokenizer.fit_on_texts(dfAllData["tweet"].values)
  X = tokenizer.texts_to_sequences(dfAllData["tweet"].values)
  X = tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=MAX_LENGTHS)
  Ylstm = tf.keras.utils.to_categorical(dfAllData["labels"])
  #x_trainlstm,x_valLstm,y_trainlstm,y_valLstm = train_test_split(X,Ylstm,test_size=0.2,random_state=1)
  #return x_trainlstm,x_valLstm,y_trainlstm,y_valLstm
  return X,Ylstm
"""from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = WORD_SIZE)
tokenizer.fit_on_texts(dfAllData["tweet"].values)
X = tokenizer.texts_to_sequences(dfAllData["tweet"].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X)
Ylstm = tf.keras.utils.to_categorical(dfAllData["labels"])
#x_trainlstm,x_valLstm,y_trainlstm,y_valLstm = train_test_split(X,Ylstm,test_size=0.001,random_state=1)
tokenizer.sequences_to_texts([x_trainlstm[0]])
#return x_trainlstm,x_valLstm,y_trainlstm,y_valLstm"""

In [ ]:
#Not Using its Attention to train.


def dataforIndoBert():
  tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

  token_train = {
      "tweet" : tokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  data = {
    "tweet" : tf.squeeze(tf.convert_to_tensor(token_train["tweet"]["input_ids"])),
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return data["tweet"].numpy(),Y


def dataforRoberta():
  RobertaModelName = "cahya/roberta-base-indonesian-522M"
  RobertaTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_Roberta = {
      "tweet" : RobertaTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_Roberta["tweet"]["input_ids"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataRoberta["tweet"].numpy(),Y


def dataforBert():
  BertModelName = "cahya/bert-base-indonesian-522M"
  BertModelTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_bert = {
      "tweet" : BertModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  train_databert = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_bert["tweet"]["input_ids"]))
  }


  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_databert["tweet"].numpy(),Y

def dataforGPT2():
  GPT2Modelname = "cahya/gpt2-small-indonesian-522M"
  Gpt2ModelTokenizer = AutoTokenizer.from_pretrained(GPT2Modelname)
  Gpt2ModelTokenizer.pad_token = Gpt2ModelTokenizer.eos_token
  token_train_GPT2 = {
      "tweet" : Gpt2ModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          
          truncation = True
      ),
  }

  train_dataGPT2 = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_GPT2["tweet"]["input_ids"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataGPT2["tweet"].numpy(),Y
def dataforXLMRoBERTa():

  XLMRobertaName = "jplu/tf-xlm-roberta-base"
  XLMRobertaMultiLingualTokenizer = AutoTokenizer.from_pretrained(XLMRobertaName)

  token_train_XLMRoberta = {
      "tweet" : XLMRobertaMultiLingualTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataXLMRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_XLMRoberta["tweet"]["input_ids"]))
  }

  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataXLMRoberta["tweet"].numpy(), Y

In [ ]:
#Not Using its Attention to train.

def dataforIndoBertAttention():
  tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

  token_train = {
      "tweet" : tokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  data = {
    "tweet" : tf.squeeze(tf.convert_to_tensor(token_train["tweet"]["input_ids"])),
    "tweetMask" :  tf.squeeze(tf.convert_to_tensor(token_train["tweet"]["attention_mask"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)
  data = {
      "tweet" : data["tweet"].numpy(),
      "tweetMask" : data["tweet"].numpy()
  }
  return data,Y


def dataforRobertaAttention():
  RobertaModelName = "cahya/roberta-base-indonesian-522M"
  RobertaTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_Roberta = {
      "tweet" : RobertaTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_Roberta["tweet"]["input_ids"])),
       "tweetMask" :  tf.squeeze(tf.convert_to_tensor(token_train_Roberta["tweet"]["attention_mask"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)
  train_dataRoberta = {
      "tweet" : train_dataRoberta["tweet"].numpy(),
      "tweetMask" : train_dataRoberta["tweet"].numpy()
  }
  return train_dataRoberta,Y


def dataforBertAttentions():
  BertModelName = "cahya/bert-base-indonesian-522M"
  BertModelTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_bert = {
      "tweet" : BertModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  train_databert = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_bert["tweet"]["input_ids"])),
      "tweetMask" :  tf.squeeze(tf.convert_to_tensor(token_train_bert["tweet"]["attention_mask"]))
  }


  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)
  train_databert = {
      "tweet" : train_databert["tweet"].numpy(),
      "tweetMask" : train_databert["tweet"].numpy()
  }
  return train_databert,Y

def dataforGPT2Attentions():
  GPT2Modelname = "cahya/gpt2-small-indonesian-522M"
  Gpt2ModelTokenizer = AutoTokenizer.from_pretrained(GPT2Modelname)
  Gpt2ModelTokenizer.pad_token = Gpt2ModelTokenizer.eos_token
  token_train_GPT2 = {
      "tweet" : Gpt2ModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          
          truncation = True
      ),
  }

  train_dataGPT2 = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_GPT2["tweet"]["input_ids"])),
      "tweetMask" :  tf.squeeze(tf.convert_to_tensor(token_train_GPT2["tweet"]["attention_mask"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)
  train_dataGPT2 = {
      "tweet" : train_dataGPT2["tweet"].numpy(),
      "tweetMask" : train_dataGPT2["tweet"].numpy()
  }
  return train_dataGPT2,Y
def dataforXLMRoBERTaAttention():

  XLMRobertaName = "jplu/tf-xlm-roberta-base"
  XLMRobertaMultiLingualTokenizer = AutoTokenizer.from_pretrained(XLMRobertaName)

  token_train_XLMRoberta = {
      "tweet" : XLMRobertaMultiLingualTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataXLMRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_XLMRoberta["tweet"]["input_ids"])),
      "tweetMask" :  tf.squeeze(tf.convert_to_tensor(token_train_XLMRoberta["tweet"]["attention_mask"]))
  }

  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)
  train_dataXLMRoberta = {
      "tweet" : train_dataXLMRoberta["tweet"].numpy(),
      "tweetMask" : train_dataXLMRoberta["tweet"].numpy()
  }
  return train_dataXLMRoberta, Y

In [ ]:
## Architecture
def architectureTransformers(passedModel):
  METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
  ]

  ids1 = tf.keras.layers.Input(shape=(MAX_LENGTHS,),dtype=tf.int32,name="tweet")
  #attentions = tf.keras.layers.Input(shape=(MAX_LENGTHS,),dtype=tf.int32,name="tweetMask")
  last_hidden_state, pooler_output = passedModel(ids1).to_tuple()
  layers = tf.keras.layers.Flatten()(last_hidden_state)
  layers = tf.keras.layers.Dropout(0.3)(layers)
  layers = tf.keras.layers.Dense(1024,activation=tf.nn.relu)(layers)
  layers = tf.keras.layers.Dense(128,activation=tf.nn.relu)(layers)
  layers = tf.keras.layers.Dropout(0.3)(layers)
  fin_layer = tf.keras.layers.Dense(NUM_CATEGORICAL,activation= tf.nn.softmax)(layers)

  final = tf.keras.Model(inputs=[ids1],outputs=[fin_layer])
  
  for layer in final.layers:
      layer.trainable = False
  for layer in final.layers[2:]:
    layer.trainable = True 
  optimizer = tf.keras.optimizers.Adam(learning_rate=.0001)
  final.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer,metrics = METRICS)

  return final

In [ ]:
METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),]

In [ ]:
def indoBertModelAttention():
  ids1 = tf.keras.layers.Input(shape=(MAX_LENGTHS,),dtype=tf.int32,name="tweet")
  attention = tf.keras.layers.Input(shape=(MAX_LENGTHS,),dtype=tf.int32,name="tweetMask")

  last_hidden_state, pooler_output = IndoBert(ids1,attention_mask=attention).to_tuple()
  layers = tf.keras.layers.Flatten()(last_hidden_state)
  layers = tf.keras.layers.Dense(units=500,activation=tf.nn.relu)(layers)

  layers = tf.keras.layers.Dense(128,activation=tf.nn.relu)(layers)
  layers = tf.keras.layers.Dropout(0.3)(layers)
  fin_layer = tf.keras.layers.Dense(NUM_CATEGORICAL,activation= tf.nn.softmax)(layers)

  final = tf.keras.Model(inputs=[ids1,attention],outputs=[fin_layer])
  
  for layer in final.layers:
      layer.trainable = False
  for layer in final.layers[2:]:
    layer.trainable = True 
  optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
  final.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer,metrics = METRICS)

  return final

In [ ]:
## LSTM MODELS basic Attentions
RNN_CELL_SIZE = 32
class Attention(tf.keras.Model):
  def __init__(self,units):
    super(Attention,self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  
  def call(self,features,hidden):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    score = tf.nn.tanh(
        self.W1(features) + self.W2(hidden_with_time_axis)
    )
    attention_weights = tf.nn.softmax(self.V(score), axis = 1)

    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis= 1)
    return context_vector, attention_weights
def attentionLayer():
  sequence_input = tf.keras.Input(shape=(MAX_LENGTHS,),dtype=tf.int32)
  embedded_sequences = tf.keras.layers.Embedding(WORD_SIZE, MAX_LENGTHS)(sequence_input)
  lstmLayer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(RNN_CELL_SIZE, return_sequences=True),name= "bi_lstm")(embedded_sequences)
  (lstm,forward_h, forward_c, backward_h, backward_c) = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(RNN_CELL_SIZE,return_state=True), name="bi_lstm_1")(lstmLayer)
  state_h = tf.keras.layers.Concatenate()([forward_h,backward_h])
  state_c = tf.keras.layers.Concatenate()([forward_c,backward_c])
  context_vector ,attention_weights = Attention(10)(lstm,state_h)
  dense = tf.keras.layers.Dense(20 ,activation = tf.nn.relu)(context_vector)
  dense = tf.keras.layers.Dropout(0.10)(dense)
  output = tf.keras.layers.Dense(4, activation = tf.nn.softmax)(dense)

  model = tf.keras.Model(inputs = [sequence_input], outputs = output)
  print(model.summary())
  tf.keras.utils.plot_model(model, show_shapes=True, dpi=90)
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=METRICS)
  return model
#x_dataLstm,x_valLstm,y_dataLstm,y_valLstm = dataForLstm()
#model = attentionLayer()
#EPOCHS = 5
#history = model.fit(x_dataLstm,y_dataLstm,
#                    batch_size=64,
#                    epochs=EPOCHS,
#                    validation_split=0.2)

def lstmModel(X):
  input_lstm = tf.keras.layers.Input(shape=(X,),dtype = tf.int32,name="tweet")
  layers  = tf.keras.layers.Embedding(WORD_SIZE,23 )(input_lstm)
  layers = tf.keras.layers.SpatialDropout1D(0.4)(layers)
  layers = tf.keras.layers.LSTM(64, activation=tf.nn.tanh,dropout = 0.4,recurrent_dropout=0.2,)(layers)
  layers = tf.keras.layers.Dense(128,activation=tf.nn.relu)(layers)
  output = tf.keras.layers.Dense(NUM_CATEGORICAL,activation=tf.nn.softmax)(layers)
  
  model = tf.keras.Model(inputs= [input_lstm], outputs = [output])

  model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer= tf.keras.optimizers.Adam(0.001),
                metrics = METRICS)
  return model

def lstmBidirectional(X):
  input_lstm = tf.keras.layers.Input(shape=(X,),dtype = tf.int32,name="tweet")
  layers  = tf.keras.layers.Embedding(WORD_SIZE,23 )(input_lstm)
  layers = tf.keras.layers.SpatialDropout1D(0.4)(layers)
  layers = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation=tf.nn.tanh,dropout = 0.4,recurrent_dropout=0.2))(layers)
  layers = tf.keras.layers.Dense(128,activation=tf.nn.relu)(layers)
  output = tf.keras.layers.Dense(NUM_CATEGORICAL,activation=tf.nn.softmax)(layers)
  
  model = tf.keras.Model(inputs= [input_lstm], outputs = [output])

  model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer= tf.keras.optimizers.Adam(0.001),
                metrics = METRICS)
  return model
#x_trainlstm,x_valLstm,y_trainlstm,y_valLstm = dataForLstm()
#model = lstmModel(142)
#EPOCHS = 10
#history = model.fit(x=x_trainlstm,y=y_trainlstm ,validation_data = (x_valLstm,y_valLstm),batch_size=64,epochs=EPOCHS)

In [ ]:
"""pred = model.predict(X)
def getDifferTrainingLSTM(dat,y_train,pred):
  y_train = np.argmax(y_train,axis=1)
  pred = np.argmax(pred,axis=1)
  print(pred)
  differ = []
  for indx,content in enumerate(dat[:,0]):
     if (y_train[indx] != pred[indx]):
       differ.append( (indx,tokenizer.sequences_to_texts(dat[[indx]]),y_train[indx],pred[indx] ))
  return differ
pd.DataFrame(getDifferTrainingLSTM(X,Ylstm,pred)).to_excel("Klasifikasi yang SalahLSTM.xlsx",index=False)
"""

In [ ]:
"""def changeTheLabels(df1,df2):
  for i in (df2.loc[:,3]):
    print(i)
    if (df1.loc[i,"labels"] != df2.loc[i,3]):
      df1.loc[i,"labels"] = df2.loc[i,3]
  print("changed Succesfully blok")
copyDfAllData = dfAllData.copy()
changeTheLabels(copyDfAllData,checkitBrot)"""

In [ ]:
"""copyDfAllData.to_csv("DataSwapped.csv",index=False)"""

In [ ]:
"""checkitBrot = pd.read_excel("Klasifikasi yang SalahLSTM.xlsx")
checkitBrot"""

In [ ]:
def initLstm(X):
    return [lstmModel(X),lstmBidirectional(X)]

In [ ]:
FinalHistory_lstm = []
trainedLstmModels = []
splits = 5
EPOCHS = 8

def TrainLSTModels(splits,EPOCHS):
    from sklearn.model_selection import StratifiedShuffleSplit
    skf = StratifiedShuffleSplit(n_splits=splits,random_state=10)
    modelNames = ["LSTM","LSTM Bidirectional"]
    lstm,lstmBd = initLstm(MAX_LENGTHS)
    lstmModels = [lstm,lstmBd]
    Xdat,Ydat = dataForLstm()
    fold = 1
    for modelname, model in zip(modelNames,lstmModels):
        HISTORY_LSTM = []
        for train_indx, val_indx in skf.split(Xdat,np.argmax(Ydat,axis=1)):
            print("============================================")
            print(f"TRAINING {modelname} : , {fold}" )
            if(modelname == "LSTM"):
              model = lstmModel(MAX_LENGTHS)
            elif(modelname == "LSTM Bidirectional"):
              model = lstmBidirectional(MAX_LENGTHS)
            tf.keras.backend.clear_session()
            train = Xdat[train_indx]
            val = Xdat[val_indx]
            ytrain = Ydat[train_indx] 
            yval = Ydat[val_indx]
            trainData = (tf.data.Dataset.from_tensor_slices((train,ytrain))).batch(64).prefetch(AUTO)
            valData = (tf.data.Dataset.from_tensor_slices((val,yval))).batch(64).prefetch(AUTO).cache()
            history = model.fit( (trainData),validation_data = valData,epochs=EPOCHS)
            #print(f"Scores For fold {fold} ACC {history.history['accuracy'].mean()} , VAL ACC {history.history['val_accuracy'].mean()}")
            data = ("rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","f1ScoreTrain","f1ScoreVal")
            datBang = [ (np.mean(history.history["accuracy"]), np.mean(history.history["val_accuracy"]),
                         np.mean(history.history["loss"]), np.mean(history.history["val_loss"]),
                         np.mean(history.history["precision"]), np.mean(history.history["val_precision"]),
                         np.mean(history.history["recall"]), np.mean(history.history["val_recall"]) )  ]
            HISTORY_LSTM.append(datBang)
            fold += 1
        hist = np.array(HISTORY_LSTM,dtype=np.float32).reshape(splits,8)
        data = (hist[:,0].mean(),hist[:,1].mean(),hist[:,2].mean(),
                hist[:,3].mean(),hist[:,4].mean(),hist[:,5].mean(),
                hist[:,6].mean(),hist[:,7].mean(),modelname)
        FinalHistory_lstm.append((data))
        trainedLstmModels.append(model)
TrainLSTModels(splits,EPOCHS)

In [ ]:
dfResultLstm = pd.DataFrame(FinalHistory_lstm,columns=[ "rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","ModelName"])
dfResultLstm.to_excel("ResultLstmMerged.xlsx",index=False)

In [ ]:
dfResultLstm

In [ ]:
# Using its Attention to train.

def dataforIndoBert():
  tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

  token_train = {
      "tweet" : tokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  data = {
    "tweet" : tf.squeeze(tf.convert_to_tensor(token_train["tweet"]["input_ids"])),
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return data["tweet"].numpy(),Y


def dataforRoberta():
  RobertaModelName = "cahya/roberta-base-indonesian-522M"
  RobertaTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_Roberta = {
      "tweet" : RobertaTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_Roberta["tweet"]["input_ids"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataRoberta["tweet"].numpy(),Y


def dataforBert():
  BertModelName = "cahya/bert-base-indonesian-522M"
  BertModelTokenizer = AutoTokenizer.from_pretrained(RobertaModelName)
  token_train_bert = {
      "tweet" : BertModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }
  train_databert = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_bert["tweet"]["input_ids"]))
  }


  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_databert["tweet"].numpy(),Y

def dataforGPT2():
  GPT2Modelname = "cahya/gpt2-small-indonesian-522M"
  Gpt2ModelTokenizer = AutoTokenizer.from_pretrained(GPT2Modelname)
  Gpt2ModelTokenizer.pad_token = Gpt2ModelTokenizer.eos_token
  token_train_GPT2 = {
      "tweet" : Gpt2ModelTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          
          truncation = True
      ),
  }

  train_dataGPT2 = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_GPT2["tweet"]["input_ids"]))
  }
  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataGPT2["tweet"].numpy(),Y
def dataforXLMRoBERTa():

  XLMRobertaName = "jplu/tf-xlm-roberta-base"
  XLMRobertaMultiLingualTokenizer = AutoTokenizer.from_pretrained(XLMRobertaName)

  token_train_XLMRoberta = {
      "tweet" : XLMRobertaMultiLingualTokenizer.batch_encode_plus(
          dfAllData["tweet"].to_list(),
          max_length = MAX_LENGTHS,
          pad_to_max_length = True,
          truncation = True
      ),
  }

  train_dataXLMRoberta = {
      "tweet" : tf.squeeze(tf.convert_to_tensor(token_train_XLMRoberta["tweet"]["input_ids"]))
  }

  Y = tf.convert_to_tensor(dfAllData["labels"])
  Y = to_categorical(Y,NUM_CATEGORICAL)

  return train_dataXLMRoberta["tweet"].numpy(), Y

In [ ]:

IndoBert = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2")

RobertaModelName = "cahya/roberta-base-indonesian-522M"
RobertaModel = TFAutoModel.from_pretrained(RobertaModelName)

BertModelName = "cahya/bert-base-indonesian-522M"
BertModel = TFAutoModel.from_pretrained(BertModelName)

XLMRobertaName = "jplu/tf-xlm-roberta-base"
XLMRobertaMultiLingualModel = TFAutoModel.from_pretrained(XLMRobertaName)

GPT2Modelname = "cahya/gpt2-small-indonesian-522M"
Gpt2Model = TFAutoModel.from_pretrained(GPT2Modelname)

def indoBertModel():
  return architectureTransformers(IndoBert)

def robertaModel():
  model = architectureTransformers(RobertaModel)
  return model

def bertModel():
  model = architectureTransformers(BertModel)
  return model

def GPT2Model():
  return architectureTransformers(Gpt2Model)

def XlmRoberta():
  return architectureTransformers(XLMRobertaMultiLingualModel)

def initModel():
  return bertModel(),indoBertModel(),robertaModel(),XlmRoberta(),GPT2Model()

In [ ]:
FinalHistory_Transformers = []
trainedTransformerModels = []
splits = 5
EPOCHS = 8

def TrainAttentions(splits,EPOCHS):
  from sklearn.model_selection import StratifiedShuffleSplit
  skf = StratifiedShuffleSplit(n_splits=splits)

  fold = 1
  #y_train= np.argmax(y_train,axis=1)
  historyList = [] 
  bertmodelObj,indobertModelObj,robertaModelObj,xlmRobertaModelObj,GPT2ModelObj = initModel()
  modelNames = ["IndoBert","Roberta","Bert","GPT2","XLMROBERTA"]
  modelTransformers = [indobertModelObj,robertaModelObj,bertmodelObj,GPT2Model,xlmRobertaModelObj]
  dataTransformers = [dataforIndoBert(),dataforRoberta(),dataforBert(),dataforGPT2(),dataforXLMRoBERTa()]
  #modelNames = ["XLMROBERTA"]
  #modelTransformers = [xlmRobertaModelObj]
  #dataTransformers = [dataforXLMRoBERTa()]
  for modelname, model,content in zip(modelNames,modelTransformers,dataTransformers):
    HISTORY_TRANSFORMERS = []
    for train_indx, val_indx in skf.split(content[0],np.argmax(content[1],axis=1)):
      print("============================================")
      print(f"TRAINING {modelname} : , {fold}" )
      if (modelname == "IndoBert"):
        model = indoBertModel()
      elif (modelname=="Roberta"):
        model = robertaModel()
      elif (modelname == "Bert"):
        model = bertModel()
      elif (modelname == "GPT2"):
        model = GPT2Model()
      elif (modelname == "XLMROBERTA"):
        model = XlmRoberta()
      train = content[0][train_indx]
      val = content[0][val_indx]

      ytrain = content[1][train_indx] 
      yval = content[1][val_indx]
      trainData = (tf.data.Dataset.from_tensor_slices((train,ytrain))).batch(64).prefetch(AUTO)
      valData = (tf.data.Dataset.from_tensor_slices((val,yval))).batch(64).prefetch(AUTO).cache()
      history = model.fit( (trainData),validation_data = valData,epochs=EPOCHS)
      data = ("rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","f1ScoreTrain","f1ScoreVal")
      datBang = [ (np.mean(history.history["accuracy"]), np.mean(history.history["val_accuracy"]),
                         np.mean(history.history["loss"]), np.mean(history.history["val_loss"]),
                         np.mean(history.history["precision"]), np.mean(history.history["val_precision"]),
                         np.mean(history.history["recall"]), np.mean(history.history["val_recall"]) )  ]
      HISTORY_TRANSFORMERS.append(datBang)
      fold += 1
    hist = np.array(HISTORY_TRANSFORMERS,dtype=np.float32).reshape(splits,EPOCHS)
    data = (hist[:,0].mean(),hist[:,1].mean(),hist[:,2].mean(),
                hist[:,3].mean(),hist[:,4].mean(),hist[:,5].mean(),
                hist[:,6].mean(),hist[:,7].mean(),modelname)
    FinalHistory_Transformers.append((data))
    trainedTransformerModels.append(model)
TrainAttentions(splits,EPOCHS)

In [ ]:
dfResultTransformer =  pd.DataFrame(FinalHistory_Transformers,columns=[ "rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","ModelName"])

dfResultTransformer.to_excel("ResultTransformerMerged.xlsx",index=False)

In [ ]:
dfResultTransformer

In [ ]:
step = len(token_train_XLMRoberta["tweet"]["input_ids"]) // 64
def initModelAttentions():
  dataForIndoBert = 
def trainBert(EPOCHS,CALLBACKS):
  history = bertmodelObj.fit(train_databert,steps_per_epoch = step, validation_data=validation_databert,epochs=EPOCHS)
  return history

def trainRoberta(EPOCHS,CALLBACKS):
  history = robertaModelObj.fit(train_dataRoberta,steps_per_epoch = step,validation_data=validation_dataRoberta,epochs=EPOCHS)
  return history

def trainXlmRoberta(EPOCHS, CALLBACKS):
  history = xlmRobertaModelObj.fit(train_dataXLMRoberta,steps_per_epoch = step,validation_data = validation_dataXLMRoberta,epochs=EPOCHS)
  return history

def trainGpt2(EPOCHS, CALLBACKS):
  history = GPT2ModelObj.fit(train_dataGPT2,steps_per_epoch = step, validation_data = validation_dataGPT2,epochs = EPOCHS)
  return history

def trainIndoBert(EPOCHS, CALLBACKS):
  history = indobertModelObj.fit(datasetTraining,steps_per_epoch = step, validation_data = datasetValidation, epochs= EPOCHS)
  return history

In [ ]:
print(y_train)
pred =np.argmax(indobertModelObj.predict(datasetTraining,steps=step),axis=1)


In [ ]:
y_trainCopy = np.argmax(y_train.copy(),axis=1)
y_trainCopy

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_trainCopy[:5888],pred)

In [ ]:
def getDifferTrainingIndoBert():
  differ = []
  for indx,content in enumerate(x_train.loc[:5888,"tweet"].values):
     if (y_trainCopy[indx] != pred[indx]):
       differ.append( (x_train.iloc[indx,0],f"TRUE : {y_trainCopy[indx]}",f"PRED {pred[indx]}" ))
  return differ
pd.DataFrame(getDifferTrainingIndoBert()).to_excel("Klasifikasi yang SalahIndoBert.xlsx",index=False)

In [ ]:
def getDifferTrainingLSTM():
  differ = []
  for indx,content in enumerate(x_trainlstm.loc[:,"tweet"].values):
     if (y_trainlstm[indx] != pred[indx]):
       differ.append( (x_train.x_trainlstm[indx,0],f"TRUE : {y_trainlstm[indx]}",f"PRED {pred[indx]}" ))
  return differ
pd.DataFrame(getDifferTrainingIndoBert()).to_excel("Klasifikasi yang SalahLSTM.xlsx",index=False)

In [ ]:
def plotlah(history,modelName):
  import plotly.graph_objects as go
  fig = go.Figure()
  history.history["epochs"] = list(range(1,10))
  fig.add_trace(go.Scatter(y= history.history["accuracy"],x= history.history["epochs"],mode="lines+markers",name="Training acc"))
  fig.add_trace(go.Scatter(y= history.history["val_accuracy"],x= history.history["epochs"],mode="lines+markers",name="Val acc"))
  fig.update_layout(title=modelName)
  fig.show()

In [ ]:
### NOn Ettention
lstmHistory = lstm.fit(x_trainlstm,y_trainlstm,epochs=10,
                       validation_data=(x_valLstm,y_valLstm))

In [ ]:
plotlah(bertHistory,"IndoBert")

In [ ]:
plotlah(lstmHistory,"LSTM WITHOUT GRU")

In [ ]:
lstm_pred = np.argmax(lstm.predict(x_trainlstm),axis=1)
y_trainlstmRill = np.argmax(y_trainlstm,axis=1)

def confusionGelo(y_true,ypred):
  print(confusion_matrix(y_true,ypred))
print("ConfusionMatrixLSTM")
confusionGelo(y_trainlstmRill,lstm_pred)

In [ ]:
print("ConfusionMatrixTransformers")
confusionGelo(np.argmax(y_train[:5888],axis=1),pred)

In [ ]:
FinalHistory_Transformers = []
trainedTransformerModels = []
splits = 5
EPOCHS = 8
def TrainAttentions(splits,EPOCHS):
  from sklearn.model_selection import StratifiedKFold
  skf = StratifiedKFold(n_splits=splits)
  fold = 1
  #y_train= np.argmax(y_train,axis=1)
  historyList = [] 
  bertmodelObj,indobertModelObj,robertaModelObj,xlmRobertaModelObj,GPT2ModelObj = initModel()
  #modelNames = ["IndoBert","Roberta","Bert","GPT2","XLMROBERTA"]
  #modelTransformers = [indobertModelObj,robertaModelObj,bertmodelObj,GPT2Model,xlmRobertaModelObj]
  #dataTransformers = [dataforIndoBertAttention(),dataforRobertaAttention(),dataforBertAttentions(),dataforGPT2Attentions(),dataforXLMRoBERTaAttention()]
  modelNames = ["XLMROBERTA"]
  modelTransformers = [xlmRobertaModelObj]
  dataTransformers = [dataforXLMRoBERTaAttention()]
  for modelname, model,content in zip(modelNames,modelTransformers,dataTransformers):
    HISTORY_TRANSFORMERS = []
    for train_indx, val_indx in skf.split(content[0]["tweet"],np.argmax(content[1],axis=1)):
      print("============================================")
      print(f"TRAINING {modelname} : , {fold}" )
      if (modelname == "IndoBert"):
        model = indoBertModel()
      elif (modelname=="Roberta"):
        model = robertaModel()
      elif (modelname == "Bert"):
        model = bertModel()
      elif (modelname == "GPT2"):
        model = GPT2Model()
      elif (modelname == "XLMROBERTA"):
        model = XlmRoberta()
      train = content[0]["tweet"][train_indx]
      trainMask = content[0]["tweetMask"][train_indx]
      val = content[0]["tweet"][val_indx]
      valMask = content[0]["tweetMask"][val_indx]
      dataTrain = {
          "tweet" : train,
          "tweetMask" :trainMask 
      }
      dataVal = {
          "tweet" : val,
          "tweetMask" : valMask
      }
      ytrain = content[1][train_indx] 
      yval = content[1][val_indx]
      trainData = (tf.data.Dataset.from_tensor_slices((dataTrain,ytrain))).batch(64).prefetch(AUTO)
      valData = (tf.data.Dataset.from_tensor_slices((dataVal,yval))).batch(64).prefetch(AUTO).cache()
      history = model.fit( (trainData),validation_data = valData,epochs=EPOCHS)
      data = ("rataAccTrain", "rataAccVal",
                    "rataLossTrain", "rataLossVal",
                    "precisionTrain","precisionVal",
                    "recallTrain","recallVal","f1ScoreTrain","f1ScoreVal")
      datBang = [ (np.mean(history.history["accuracy"]), np.mean(history.history["val_accuracy"]),
                         np.mean(history.history["loss"]), np.mean(history.history["val_loss"]),
                         np.mean(history.history["precision"]), np.mean(history.history["val_precision"]),
                         np.mean(history.history["recall"]), np.mean(history.history["val_recall"]) )  ]
      HISTORY_TRANSFORMERS.append(datBang)
      fold += 1
    hist = np.array(HISTORY_TRANSFORMERS,dtype=np.float32).reshape(splits,EPOCHS)
    data = (hist[:,0].mean(),hist[:,1].mean(),hist[:,2].mean(),
                hist[:,3].mean(),hist[:,4].mean(),hist[:,5].mean(),
                hist[:,6].mean(),hist[:,7].mean(),modelname)
    FinalHistory_Transformers.append((data))
    trainedTransformerModels.append(model)
TrainAttentions(splits,EPOCHS)

In [ ]:
WORD_SIZE = 5000
X,Y = dataForLstm()
model = lstmModel(50)
EPOCHS = 8
history = model.fit(x=X,y=Y ,validation_split = 0.20,batch_size=64,epochs=EPOCHS)

In [ ]:
!gdown --id 1qsgKjgJhG1NO0kmUbg5LvGf0SNXckPoU
def dataForLstmtest():
  from tensorflow.keras.preprocessing.text import Tokenizer
  tokenizer = Tokenizer(num_words = WORD_SIZE)
  tokenizer.fit_on_texts(dfAllData["tweet"].values)
  X = tokenizer.texts_to_sequences(dfAllData["tweet"].values)
  X = tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=MAX_LENGTHS)
  Ylstm = tf.keras.utils.to_categorical(dfAllData["labels"])
  return tokenizer
tokenizerTest = dataForLstmtest()
test = pd.read_excel("datatest.xlsx")
test[test["label"] == 3]["label"] = 2
test["text"] = test["text"].apply(cleanIt)
test["text"] = test["text"].apply(stopWord)
test= tokenizerTest.texts_to_sequences(test["text"])
test = tf.keras.preprocessing.sequence.pad_sequences(test,maxlen=MAX_LENGTHS)
pred = np.argmax(model.predict(test),axis=1)

pred